In [1]:
import pandas as pd
import numpy as np
import glob
import datetime

In [2]:
def read_in_data(location, path):
    files = glob.glob(path + location + "_agg/*")
    df = pd.read_csv(files[0])
    filename = files[0].split("agg")[-1][1:-4]
    df["file"] = filename
    for file in files[1:]:
        new_df = pd.read_csv(file)
        filename = file.split("agg")[-1][1:-4]
        new_df["file"] = filename
        df = df.append(new_df)
    return df

In [3]:
path = "E:/Data/Monitor Data/" # The directory of wrist_agg, hip_agg, chest_agg, thigh_agg, and processed directories

df_wrist = read_in_data("wrist", path)
df_hip = read_in_data("hip", path)
df_chest = read_in_data("chest", path)
df_thigh = read_in_data("thigh", path)

groundtruth = pd.read_csv(path + "processed/groundtruth.csv")
groundtruth.drop("Unnamed: 0", axis=1, inplace=True)

In [4]:
DO_log_final_df = pd.read_csv(path + 'processed/DO_log_final.csv')

In [5]:
def datetime_filter(location, data, log):
#     log = log[log['type'] == location[0].upper()]
    
    log['date'] = pd.to_datetime(log[['start_day','start_month','start_year']]
                   .astype(str).apply(' '.join, 1), format='%d %m %Y').astype(str)
    log['start_time'] = pd.to_datetime(log['start_time'], format = '%H:%M:%S').apply(lambda x: x.time())
    log['stop_time'] = pd.to_datetime(log['stop_time'], format = '%H:%M:%S').apply(lambda x: x.time())
    
    data['actual_datetime'] = pd.to_datetime(data['actual_datetime'], format = '%Y-%m-%d %H:%M:%S')
    data['actual_date'] = data['actual_datetime'].apply(lambda x: x.date()).astype(str)
    data['actual_time'] = data['actual_datetime'].apply(lambda x: x.time())
    
    joined = pd.merge(data, log, how = 'left', left_on = 'actual_date', right_on = 'date')
    
    joined = joined[(joined['actual_time'] >= joined['start_time']) & 
                    (joined['actual_time'] <= joined['stop_time'])]
    
    return joined[['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z', 'actual_datetime', 'file']]

In [6]:
df_hip = datetime_filter('hip', df_hip, DO_log_final_df)
df_wrist = datetime_filter('wrist', df_wrist, DO_log_final_df)

In [7]:
df_chest.rename(columns = {"Accel X (g)": "Accelerometer X", 
                           "Accel Y (g)": "Accelerometer Y", 
                           "Accel Z (g)": "Accelerometer Z"}, inplace=True)
df_thigh.rename(columns = {"Accel X (g)": "Accelerometer X", 
                           "Accel Y (g)": "Accelerometer Y", 
                           "Accel Z (g)": "Accelerometer Z"}, inplace=True)

In [8]:
dt_vars = ["time", "actual_time", "start.time"]
for var in dt_vars:
    groundtruth[var] = pd.to_datetime(groundtruth[var])

In [9]:
# The time and actual_time variables give conflicting accounts of the time
display(groundtruth["time"].iloc[45:50].dt.time)
display(groundtruth["actual_time"].iloc[45:50].dt.time)

45    18:30:00
46    18:30:00
47    18:31:00
48    18:31:00
49    18:31:00
Name: time, dtype: object

45    11:30:58
46    11:30:59
47    11:31:00
48    11:31:01
49    11:31:02
Name: actual_time, dtype: object

In [10]:
groundtruth["time"] = groundtruth["time"] + pd.to_timedelta(groundtruth["actual_time"].dt.second, unit="s")
groundtruth["time"].head()

0   2017-08-02 18:30:13
1   2017-08-02 18:30:14
2   2017-08-02 18:30:15
3   2017-08-02 18:30:16
4   2017-08-02 18:30:17
Name: time, dtype: datetime64[ns]

In [11]:
groundtruth.dtypes

time                     datetime64[ns]
id                               object
coding                           object
primary_behavior                 object
primary_posture                  object
primary_upperbody                object
primary_intensity                object
secondary_behavior               object
secondary_posture                object
secondary_upperbody              object
secondary_intensity              object
num_postures                      int64
transition                        int64
actual_time              datetime64[ns]
posture_coding                   object
type                             object
start.time               datetime64[ns]
broad_activity                   object
detailed_activity                object
walking_running_bouts            object
updated_activity                 object
act_type                         object
act_type_broad                   object
dtype: object

In [12]:
kept_merge_variables = ["id", "time", "primary_behavior", "primary_posture", "primary_upperbody", 
                        "primary_intensity", "secondary_behavior", "secondary_posture", "secondary_upperbody", 
                        "secondary_intensity", "num_postures", "transition", "actual_time", "type", "start_time",
                        "broad_activity", "detailed_activity", "updated_activity", "act_type", "act_type_broad"]

dropped_merge_variables = ["coding", # Duplicates primary_intensity
                           "posture_coding", # Duplicates primary_posture
                           "walking_running_bouts" # NaNs
                          ]

In [13]:
groundtruth.drop(dropped_merge_variables, axis=1, inplace=True)

In [14]:
df_wrist["actual_datetime"] = pd.to_datetime(df_wrist["actual_datetime"])
df_hip["actual_datetime"] = pd.to_datetime(df_hip["actual_datetime"])
df_chest["actual_datetime"] = pd.to_datetime(df_chest["actual_datetime"])
df_thigh["actual_datetime"] = pd.to_datetime(df_thigh["actual_datetime"])

In [15]:
dfs = [df_wrist, df_hip, df_chest, df_thigh]
locations = ["Wrist", "Hip", "Chest", "Thigh"]
for i in range(len(dfs)):
    print(locations[i] + ":")
    display(dfs[i][dfs[i]["actual_datetime"].dt.month == 10]["actual_datetime"].dt.date.unique())

Wrist:


array([datetime.date(2017, 10, 3), datetime.date(2017, 10, 6),
       datetime.date(2017, 10, 7), datetime.date(2017, 10, 11),
       datetime.date(2017, 10, 13), datetime.date(2017, 10, 24),
       datetime.date(2017, 10, 27)], dtype=object)

Hip:


array([datetime.date(2017, 10, 3), datetime.date(2017, 10, 6),
       datetime.date(2017, 10, 7), datetime.date(2017, 10, 11),
       datetime.date(2017, 10, 13), datetime.date(2017, 10, 24),
       datetime.date(2017, 10, 27)], dtype=object)

Chest:


array([datetime.date(2017, 10, 4), datetime.date(2017, 10, 6),
       datetime.date(2017, 10, 7), datetime.date(2017, 10, 13),
       datetime.date(2017, 10, 14), datetime.date(2017, 10, 8),
       datetime.date(2017, 10, 11), datetime.date(2017, 10, 25),
       datetime.date(2017, 10, 27)], dtype=object)

Thigh:


array([datetime.date(2017, 10, 4), datetime.date(2017, 10, 6),
       datetime.date(2017, 10, 7), datetime.date(2017, 10, 13),
       datetime.date(2017, 10, 14), datetime.date(2017, 10, 8),
       datetime.date(2017, 10, 11), datetime.date(2017, 10, 25),
       datetime.date(2017, 10, 27)], dtype=object)

In [16]:
DO_log_final_df

,id,obs,type,start_month,start_day,start_year,start_time,stop_time,duration,session,date
0,AM01,DO2,L,10,3,2017,18:44:45,20:47:00,2:02:15,1,2017-10-03
1,AM01,DO1,H,10,6,2017,16:43:57,18:45:00,2:01:03,2,2017-10-06
2,AM02,DO1,H,7,24,2017,13:17:10,15:17:32,2:00:22,1,2017-07-24
3,AM02,DO2_a,A,7,25,2017,08:00:27,08:52:32,0:52:05,2,2017-07-25
4,AM02,DO2_b,A,7,25,2017,08:56:13,10:12:29,1:16:16,3,2017-07-25
5,AM03,DO1,L,7,25,2017,14:00:10,16:00:23,2:00:13,1,2017-07-25
6,AM03,DO2,H,7,27,2017,13:01:29,15:00:39,1:59:10,2,2017-07-27
7,AM04,DO1,W,9,12,2017,12:02:56,14:04:00,2:01:04,1,2017-09-12
8,AM04,DO2,A,9,17,2017,14:15:58,16:16:00,2:00:02,2,2017-09-17
9,AM05,DO2,L,9,19,2017,19:38:53,21:40:00,2:01:07,1,2017-09-19


In [17]:
DO_log_final_df[DO_log_final_df['type'] == 'H'][['start_month', 'start_day', 'start_year']]

,start_month,start_day,start_year
1,10,6,2017
2,7,24,2017
6,7,27,2017
12,8,2,2017
40,11,9,2017
42,12,2,2017
50,2,17,2018
53,2,24,2018
54,2,24,2018
56,5,18,2018


In [18]:
groundtruth["time"].dt.date.unique()

array([datetime.date(2017, 8, 2), datetime.date(2017, 10, 11),
       datetime.date(2018, 2, 8), datetime.date(2017, 10, 25),
       datetime.date(2017, 9, 21), datetime.date(2017, 10, 8),
       datetime.date(2017, 8, 1), datetime.date(2017, 10, 27),
       datetime.date(2018, 5, 18), datetime.date(2018, 1, 20),
       datetime.date(2018, 2, 24), datetime.date(2018, 2, 25),
       datetime.date(2017, 11, 4), datetime.date(2017, 11, 5),
       datetime.date(2017, 10, 6), datetime.date(2017, 10, 7),
       datetime.date(2017, 12, 2), datetime.date(2017, 9, 20),
       datetime.date(2017, 11, 9), datetime.date(2017, 9, 2),
       datetime.date(2017, 10, 4), datetime.date(2018, 5, 19),
       datetime.date(2017, 9, 23), datetime.date(2017, 7, 27),
       datetime.date(2017, 7, 25), datetime.date(2018, 2, 17),
       datetime.date(2018, 1, 17), datetime.date(2017, 7, 24),
       datetime.date(2018, 2, 6), datetime.date(2017, 9, 17),
       datetime.date(2017, 8, 3), datetime.date(2017, 9, 

In [19]:
# Groundtruth unique dates in October
display(groundtruth[groundtruth["time"].dt.month == 10]["time"].dt.date.unique())

array([datetime.date(2017, 10, 11), datetime.date(2017, 10, 25),
       datetime.date(2017, 10, 8), datetime.date(2017, 10, 27),
       datetime.date(2017, 10, 6), datetime.date(2017, 10, 7),
       datetime.date(2017, 10, 4), datetime.date(2017, 10, 13),
       datetime.date(2017, 10, 14)], dtype=object)

In [20]:
locations = [#"wrist", "hip", 
             "chest", "thigh"]
all_dfs = [#df_wrist, df_hip, 
           df_chest, df_thigh]

In [21]:
df_wrist.merge(groundtruth, how="inner", left_on = "actual_datetime", right_on = "time")

,Accelerometer X,Accelerometer Y,Accelerometer Z,actual_datetime,file,time,id,primary_behavior,primary_posture,primary_upperbody,...,num_postures,transition,actual_time,type,start.time,broad_activity,detailed_activity,updated_activity,act_type,act_type_broad
0,0.246100,-0.488013,-0.874450,2017-07-25 15:00:27,AG_AM02WRT (2017-07-19)AGGREGATE,2017-07-25 15:00:27,AM02,EX- jogging,LA- stand and move,unknown,...,1,0,2021-05-11 08:00:27,A-2,2017-07-25 08:00:00,mixed-activity,walking light,mixed-activity,EX-run,EX
1,0.154950,0.105163,0.947400,2017-07-25 15:00:27,AG_AM03WRT (2017-07-24)AGGREGATE,2017-07-25 15:00:27,AM02,EX- jogging,LA- stand and move,unknown,...,1,0,2021-05-11 08:00:27,A-2,2017-07-25 08:00:00,mixed-activity,walking light,mixed-activity,EX-run,EX
2,0.245900,-0.488425,-0.874550,2017-07-25 15:00:28,AG_AM02WRT (2017-07-19)AGGREGATE,2017-07-25 15:00:28,AM02,EX- jogging,LA- stand and move,unknown,...,1,0,2021-05-11 08:00:28,A-2,2017-07-25 08:00:00,mixed-activity,walking light,mixed-activity,EX-run,EX
3,0.157750,0.098412,0.949162,2017-07-25 15:00:28,AG_AM03WRT (2017-07-24)AGGREGATE,2017-07-25 15:00:28,AM02,EX- jogging,LA- stand and move,unknown,...,1,0,2021-05-11 08:00:28,A-2,2017-07-25 08:00:00,mixed-activity,walking light,mixed-activity,EX-run,EX
4,0.244950,-0.489200,-0.874900,2017-07-25 15:00:29,AG_AM02WRT (2017-07-19)AGGREGATE,2017-07-25 15:00:29,AM02,EX- jogging,LA- stand and move,unknown,...,1,0,2021-05-11 08:00:29,A-2,2017-07-25 08:00:00,mixed-activity,walking light,mixed-activity,EX-run,EX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13173,0.465862,0.674600,0.403475,2017-10-13 17:49:28,AG_AM16WRT (2017-10-06)AGGREGATE,2017-10-13 17:49:28,AM15,WRK- screen basedentary - Education and Health...,SB-sitting,typing,...,1,0,2021-05-11 10:49:28,W-15,2017-10-13 10:49:00,sit/stand,sit/lie,sit/lie,WRK-screen,WRK
13174,0.977350,-0.116325,-0.357138,2017-10-13 17:49:29,AG_AM15WRT (2017-10-09)AGGREGATE,2017-10-13 17:49:29,AM15,WRK- screen basedentary - Education and Health...,SB-sitting,typing,...,1,0,2021-05-11 10:49:29,W-15,2017-10-13 10:49:00,sit/stand,sit/lie,sit/lie,WRK-screen,WRK
13175,0.537150,0.738313,0.591200,2017-10-13 17:49:29,AG_AM16WRT (2017-10-06)AGGREGATE,2017-10-13 17:49:29,AM15,WRK- screen basedentary - Education and Health...,SB-sitting,typing,...,1,0,2021-05-11 10:49:29,W-15,2017-10-13 10:49:00,sit/stand,sit/lie,sit/lie,WRK-screen,WRK
13176,0.954037,-0.120450,-0.423363,2017-10-13 17:49:30,AG_AM15WRT (2017-10-09)AGGREGATE,2017-10-13 17:49:30,AM15,WRK- screen basedentary - Education and Health...,SB-sitting,typing,...,1,0,2021-05-11 10:49:30,W-15,2017-10-13 10:49:00,sit/stand,sit/lie,sit/lie,WRK-screen,WRK


In [22]:
new_dfs = []
for df in all_dfs:
#     df.drop("file", axis=1, inplace=True)
    merged_df = df.merge(groundtruth, how = "inner", left_on = "actual_datetime", right_on = "time")
    #Adding new variables
    merged_df["magnitude_of_change_X"] = merged_df["Accelerometer X"].diff().fillna(0)
    merged_df["magnitude_of_change_Y"] = merged_df["Accelerometer Y"].diff().fillna(0)
    merged_df["magnitude_of_change_Z"] = merged_df["Accelerometer Z"].diff().fillna(0)
    merged_df["time_diff_seconds"] = (merged_df["time"] - merged_df["time"].shift()).dt.total_seconds().fillna(1).astype(int)
    
    merged_df["year"] = merged_df["time"].dt.year
    merged_df["month"] = merged_df["time"].dt.month
    merged_df["day_of_week"] = merged_df["time"].dt.dayofweek
    merged_df["day"] = merged_df["time"].dt.day
    merged_df["hour"] = merged_df["time"].dt.hour
    merged_df["minute"] = merged_df["time"].dt.minute
    merged_df["second"] = merged_df["time"].dt.second
    
    new_dfs.append(merged_df)

In [23]:
output_dir = "E:/Data/Monitor Data/data_frames/"

for i in range(len(new_dfs)):
    new_dfs[i].to_csv(output_dir + locations[i] + ".csv", index=False)